# Converter

## How to use a converter ?

This tutorial shows the way to call converter, and in a second part we explain this architecture. <br>
So, with this converter you can convert **motor IPMSM, SPMSM, SCIM and WRSM**. <br>
This converter is between Motor-CAD and Pyleecan but the architecture is create to be reuse if you want an other converter.<br>
You can select your file Motor-CAD ".mot" directly with GUI Pyleecan, the same way as loading a json file.

This converter could propose a reversibility, but attention the reversibility is not perfect. In fact, we don't know the fonctionnement of Motor-CAD, that is why, we return a dictionnary with all equivalent parameter between Motor-CAD and Pyleecan present in a file.mot 

If you want to find equivalent, approximation or error you have a file Pyleecan.log. In this file you have big step to example equivalent between parallel_slot and slotW21. Or if you have an error or approximation in conversion. <br>

Ex log : <br>
2023-12-13 10:42:50-   INFO: Pyleecan version : 1.5.2 [Pyleecan] <br>
2023-12-13 10:42:50-   INFO: Path file use to convert : /Users\LAP17\Documents/machine_MC_P/file_mot/WRSM_salient_pole.mot [Pyleecan]<br>
2023-12-13 10:42:50-   INFO: Conversion file .mot into dict [Pyleecan]<br>
2023-12-13 10:42:50-   INFO: Conversion dict into obj machine [Pyleecan]<br>
2023-12-13 10:42:50-   INFO: Conversion SYNC into MachineWRSM [Pyleecan]<br>
2023-12-13 10:42:50-   INFO: Conversion Parallel_Tooth into SlotW11 [Pyleecan]<br>
2023-12-13 10:42:50-   INFO: Conversion AWG_Table into CondType12 [Pyleecan]<br>
2023-12-13 10:42:50-   INFO: Conversion Sync_Salient_Pole into SlotW62 [Pyleecan]<br>
2023-12-13 10:42:50-  DEBUG: No duct to convert at Rotor [Pyleecan]<br>
2023-12-13 10:42:50-   INFO: Conversion AWG_Table into CondType12 [Pyleecan]<br>
2023-12-13 10:42:50-   INFO: skew, not implemented [Pyleecan]<br>
2023-12-13 10:42:50-   INFO: End of conversion, obj machine is create [Pyleecan]

## Converter architecture

### Dictionnary

La première étape  pour simplifier le convertisseur est la création d’un dictionnaire à partir du fichier .mot.
Le fichier .mot est composé de différentes parties commençant par un titre entre crochet suivi d’une liste de valeur. Par exemple:
```yaml
[Dimensions]
Stator_Lam_Dia=130
Stator_Bore=80
Airgap=1

[Design_Options]
BPM_Rotor=Surface_Radial
Top_Bar=Round
Bottom_Bar=Rectangular 
```
 
On va donc reproduire cette architecture dans un dictionnaire, comprenant des dictionnaires comme ci-dessous.
On a reproduit la même organisation que le fichier .mot, pour simplifier l'accès aux paramètres.
```python
other_dict = {
 "[Dimensions]" : { 
"Stator_Lam_Dia" : 130,
 "Stator_Bore" : 80,
 "Airgap" : 1,
 },
 "[Design_Options]" : {
 "BPM_Rotor" : "Surface_Radial",
 "Top_Bar" : "Round",
 "Bottom_Bar" : "Rectangular",
 },
}
```

Nous avons également la gestion des unités. Lors de la création du dictionnaire équivalent au fichier .mot, un autre dictionnaire d’unité est également créé. Ce dictionnaire permettra lors de l’exécution des règles d’avoir la bonne conversion des unités. 

Ces deux dictionnaires permettront de généraliser le convertisseur à plusieurs logiciels. En effet, cette utilisation d’un dictionnaire pourra être reproduit peut importe le logiciel, le chemin écrit dans les règles seront simplement différent. 

Après avoir convertit le fichier en dictionnaire on peut débuter la conversion pour ce faire deux classes sont utilisées.


### Class presentation

#### Class Converter

In this abstract class, we have variable use to convert, and different general methods. In fact, if you want to create an other converter you can reuse this methods.

#### ConverterMC

Contrary to convert, this class has all methods specific at Motor-CAD. We have all rules, and methods convert, and particularity. And ConvertMC inherits from Convert.

![](Images/UML_convert.png)

### Rule

Pour adapter les règles aux différentes situations il existe 3 types de règle, les règles Simple, Equation et Complexe chacune régit par une classe.
Pour commencer nous avons une classe abstraite Rule, qui va permettre grâce à ces méthodes de sélectionner ou ajouter la valeur dans la machine et de la même manière dans le dictionnaire.

Pour chacune des classes RuleSimple, RuleComplex et RuleEquaion des méthodes convert_to_P et convert_to_other sont utilisées pour convertir les paramètres suivant le sens de conversion. De plus la classes RuleEquation contient une méthode solve permettant de résoudre l’équation définit dans la règle. De même pour la classe RuleComplex qui a une méthode _set_fct_name qui permet d’appeler la fonction contenant la règle.


#### Rule
In this abstract class, you can find methods to select or set value in dict or in obj machine.

#### RuleComplex

Enfin les règles Complex qui traitent les autres situations dans des fonctions spécifiques. Ces règles permettent l’appel de méthodes présentes dans Pyleecan, ce qui va simplifier la conversion. L’objectif était de limiter au maximum l’utilisation de ces règles afin de faciliter la compréhension   et aussi de réduire le temps de développement.

#### RuleSimple

Nous avons donc les règles simples qui sont présentes pour effectuer une simple conversion entre deux paramètres avec un facteur entre les deux, ainsi que la conversion de l’unité.  Par exemple, Motor-Cad paramétrise suivant les diamètres alors que Pyleecan utilise généralement les rayons. On aura donc un facteur deux pour toutes les conversions entre le diamètre et le rayon.

#### RuleEquation

Ensuite les règles Equation qui permettent de passer d'un paramètre à un autre mais en impliquant d'autres paramètres, avec la conversion de l’unité également. (ex : y = a+2b -3x). Pour résoudre plus facilement l’équation on doit respecter la règle suivante, le paramètre Motor-CAD recherché doit être « y » et le paramètre Pyleecan doit être « x ». Concernant les paramètres autour a, b, c… ces paramètres doivent être déjà définis précédemment . Par exemple, le rayon du rotor (x) correspond à un paramètre dans Pyleecan. Alors qu’il peut être retrouvé sur Motor-Cad avec le diamètre du stator (y) et la dimension de l’entrefer (a). On a donc l’équation suivante « y/2 – a = x ».

![](Images/UML_rules.png)

### Architecture presentation

L'architecture du convertisseur va suivre l'interface Pyleecan. En effet, l'interface de pyleecan est composée de différente étape qui permet de définir la machine.

![](Images/Selection_machine.png)

Par exemple pour définir la Toyota_Prius, on définit : <br>
    - Machine type  <br>
    - Machine Dimensions<br>
    - Stator Slot<br>
    - ...<br>

Dans chacune de ces étapes, nous définissons les paramètres. Dans l'exemple ci-dessous, pour la Machine Dimensions, on va définir la longueur du stator, du rotor... 

![](Images/Machine_dimension.png)

Commençons par sélectionner le type de machine.

En fonction du type de machine, nous allons appeler d'autres sélections. Ces méthodes sont équivalentes aux grandes étapes de définition d’un moteur électrique. Le schéma suivant illustre l'enchainement des sélections. Par exemple, si l’on souhaite convertir un moteur IPMSM, le convertisseur va aller chercher le type de moteur. Une fois qu’il a trouvé que c’est une IPMSM, il va appeler les étapes suivantes, comme aller sélectionner le type d’encoche. 

Pour faciliter la compréhension, l'ensemble des méthodes effectuant la sélection commence par "select". Elles sont toutes dans la classe Convert. L'objectif de ces méthodes est d'appeler les bonnes règles en fonction des besoins et d'orienter vers les autres fichiers de sélection.

Les sélections dans ces méthodes sont effectuées à partir d'objet machine de Pyleecan, pour garder la possibilité de réutiliser ces méthodes. 
Cependant, pour avoir toujours cette sélection possible, des méthodes spécifiques au convertisseur (présent dans la classe convertMC) sont nécessaires.

Ces méthodes commençant par "convert" vont permettre de définir la machine en lui ajoutant ces grandes étapes, on a par exemple le type de machine, le type de slot, le type de notch...

![](Images/schematics_architecture_converter.png)

Nous avons donc en fonction de l’élément choisi d'autres sélections (d'autres étapes) comme ci-dessus : 
L'objectif de cette sélection est d’ajouter les bonnes règles. Pour cela, il faut soit être très précis (connaissance du type de slot), soit être sûr que la règle puisse être généralisée (par exemple les dimensions pour toutes les SCIM).

Pour résumer, les fichiers commençant par « select » permettent d’appeler les bonnes règles en fonction de la machine Pyleecan. Pour pouvoir être réversibles et communs à tous les convertisseurs, ils appellent d’autres méthodes commençant par « convert » qui vont définir la machine avec le bon slot.

Nous arrivons donc au fonctionnement des règles, durant ce parcours les règles sont ajoutées dans un ordre précis dans une liste nommée rules_list.
A la fin de la sélection initiée par la méthode selection_machine_rules nous nous retrouvons avec une liste de règles et une machine possédant les bonnes classes. Par exemple, la machine sera une IPMSM, avec un slotW21 au stator et un HoleM50 au rotor. On a la bonne définition des éléments dans la machine, mais les paramètres comme les dimensions ne sont pas définis.

La dernière étape est de convertir l'ensemble des règles présent dans une liste et les ajouter dans l'objet machine. Cette conversion utilise le polymorphisme, ce qui permet l’appel de la bonne conversion suivant la classe des règles.

On a donc finalement l'ensemble des règles qui sont traitées, et implémentées dans l'objet machine. Ainsi la conversion sera terminée avec l'objet complètement défini.

Voici la structure simplifiée du convertisseur dans le sens de conversion d’un fichier MOTOR-CAD à une machine Pyleecan. 

![](Images/Architecture_Converter.png)

### Reversibility

Comme on a un objet machine, on peut le sauvegarder en .json, ou load le .json en machine.
Car le convertisseur peut effectuer les 2 sens de conversion. Et le fonctionnement du convertisseur ne change pas en fonction du sens.

On arrive donc à une réversibilité partiel, mais qui peut être facilement complété.

![](Images/convert_mot_to_dict.png)

Voici ci-dessous l'oragnisation des fichiers du convertisseur dans les différentes classes :
![](Images/structure_converter.png)